In [1]:
require('rnn')
require('hdf5')
require('optim')


f = hdf5.open("data.hdf5", "r")



train_rnn_X = f:read('train_rnn_X'):all()
train_rnn_Y = f:read('train_rnn_Y'):all()
train_X_sequence =  f:read('train_X_sequence'):all()
train_Y  = f:read('train_Y'):all()
valid_reduced_X = f:read('valid_reduced_X'):all()
valid_reduced_Y = f:read('valid_reduced_Y'):all()
windows_train = f:read('windows_train'):all()
windows_valid = f:read('windows_valid'):all()
test = f:read('test'):all()
valid_kaggle_with_spaces = f:read('valid_kaggle_with_spaces'):all()
valid_kaggle_without_spaces = f:read('valid_kaggle_without_spaces'):all()
valid_answers = f:read('valid_answers'):all()
nfeatures = (f:read('nfeatures'):all())[1]
nclasses = (f:read('nclasses'):all())[1]




f:close()

window_size = windows_train:size(2)
n_actual = nclasses

In [2]:

nclasses


2	


In [125]:
vocab_size = nfeatures
embed_dim = 15
lin_dim = 20
LT = nn.LookupTable(vocab_size, embed_dim)

lstm = nn.Sequential()
lstm:add(LT)
lstm:add(nn.SplitTable(1))
seqLSTM = nn.Sequencer(nn.LSTM(embed_dim, lin_dim))
seqLSTM:remember('both')
lstm:add(seqLSTM)
lstm:add(nn.SelectTable(-1))
lstm:add(nn.Linear(lin_dim, 1))

crit = nn.MSECriterion()

params, grad_params = lstm:getParameters()
params:rand(params:size()):csub(.5):mul(.05)
count = 0

In [129]:
lstm:forward(valid_kaggle_without_spaces[100]:sub(1,10))

0.001 *
 2.3950
[torch.DoubleTensor of size 1]



In [130]:
seqLSTM:forget()
count = 0
batch = 0
function feval(w_new)
 
    --count = count + 1
    count = math.fmod(count + 1,valid_kaggle_without_spaces:size(1)-batch)
    if count == 0 then
        count = 1
    end
    lstm:forget()
    
    last = torch.nonzero(valid_kaggle_without_spaces[count]-30)[-1]
    x_d = (valid_kaggle_without_spaces[count]):sub(1,last[1]+1)
    y_d = valid_answers:sub(count,count):double()
    
    
    grad_params:zero()
    preds = lstm:forward(x_d):double()

    loss = crit:forward(preds, y_d)
    grad_out = crit:backward(preds, y_d)
    
    lstm:backward(x_d, grad_out)
    
    if torch.norm(grad_params) > 5 then
        nn.Normalize(2):forward(grad_params)
        grad_params:mul(5)
    end
    
    return loss, grad_params
end

for i = 1,100 do
    -- train a mini_batch of batchSize in parallel
    _, fs = optim.adam(feval,params)
    if i % 10 == 0 then
        print('loss for iteration ' .. i  .. ' is ' .. fs[1] , count)
        
        -- print(sgd_params)
    end
end

loss for iteration 10 is 728.22339264476	10	


loss for iteration 20 is 527.59430334084	20	


loss for iteration 30 is 35.349699791419	30	


loss for iteration 40 is 1289.2822462341	40	


loss for iteration 50 is 61.24818281646	50	


loss for iteration 60 is 741.05792665581	60	


loss for iteration 70 is 520.98767950454	70	


loss for iteration 80 is 381.03122977372	80	


loss for iteration 90 is 328.66392503755	90	


loss for iteration 100 is 1013.4687250914	100	


In [6]:
function perplexity(input_set, output_set)
    sum = 0
    seqLSTM:forget()
    distributions = torch.Tensor(input_set:size(1),2)
    for i = 1,input_set:size(1) do
        
        a = lstm:forward(input_set:sub(i,i))
        
        distribution = (nn.LogSoftMax():forward(a[1]))
        
        distributions[i] = torch.exp(distribution)
        answer = distribution[output_set[i]]
        sum = sum + answer
        
    end
    return torch.exp(-sum/input_set:size(1)), distributions
end

In [36]:
a,b = perplexity((valid_reduced_X:sub(1,100000)), valid_reduced_Y)
print(a)


1.1483594744866	


In [37]:
a,b = perplexity((train_X_sequence:sub(1,1000)), train_Y:sub(1,10000))
print(a)


1.1832836537872	


In [54]:
torch.Tensor{1}

 1
[torch.DoubleTensor of size 1]



In [103]:
function predict(input_set, output_set)
    kaggle = torch.zeros(input_set:size(1))
    for i = 1,input_set:size(1) do
        last = torch.nonzero(input_set[i]-30)[-1]
        x_d = (input_set[i]):sub(1,last[1]+1)
        kaggle[i] = lstm:forward(x_d):double()
        print(lstm:forward(x_d))
    end
    if output_set ~= 0 then
        out =output_set:double()
        MSE = crit:forward(kaggle, out)
    else
        out = 0
        MSE = 0
    end
    
    return kaggle, out, MSE
end

In [104]:
k, o, mse = predict(valid_kaggle_without_spaces:sub(1,10), valid_answers:sub(1,10))


 18.5111
[torch.DoubleTensor of size 1]



 18.5111
[torch.DoubleTensor of size 1]



 18.5111
[torch.DoubleTensor of size 1]



 18.5111
[torch.DoubleTensor of size 1]



 18.5111
[torch.DoubleTensor of size 1]



 18.5111
[torch.DoubleTensor of size 1]



 18.5111
[torch.DoubleTensor of size 1]



 18.5111
[torch.DoubleTensor of size 1]



 18.5111
[torch.DoubleTensor of size 1]



 18.5111
[torch.DoubleTensor of size 1]



In [102]:
k,o

 18.5111
 18.5111
 18.5111
 18.5111
 18.5111
 18.5111
 18.5111
 18.5111
 18.5111
 18.5111
[torch.DoubleTensor of size 10]

 13
 26
 20
 20
 18
 11
 26
 34
 19
 27
[torch.DoubleTensor of size 10]



In [214]:
k,_,_ = predict(test, 0, .3)

In [218]:
function write2file(scores, fname)
    f = io.open(fname, "w")
    f:write("ID,Count\n")
    for i=1,scores:size(1) do
        s = tostring(i) .. "," .. tostring(scores[i])
        f:write(s .. "\n")
    end
    f:close()
end

write2file(k, 'preds.txt')

In [216]:
k

  4
 34
 27
 35
 24
 15
 22
  5
  6
 30
 21
  8
 17
 26
 36
 12
 10
  9
 21
 25
  8
 33
 25
 15
 19
 11
 26
 19
  9
  4
 19
 42
  6
 25
 25
 20
 16
 15
 33
 19
 29
 11
 30
 12
  6
 32
 20
 26
 10
  3
 15
 17
 22
 14
 20
 43
 22
 25
  6
 19
 18
 16
 19
 26
 27
 13
 18
 16
 32
 29
 11
 11
 18
 22
  3
 23
 15
 24
 14
 28
 13
 15
 21
 10
 32
 18
 22
 20
  7
  3
 19
 24
 32
 20
 34
 25
 31
 35
 30
  9
  5
 22
 17
 37
 29
 18
 28
 24
 53
 25
 27
 11
 39
 22
 27
 45
 26
 28
 19
 13
 20
 15
 15
 32
 12
 13
 28
 14
  9
 18
 25
 16
 17
 31
 17
 27
 26
  6
  4
  6
 12
 26
 18
  6
 12
 16
 29
 27
 22
 21
  7
 22
 18
 13
 17
 18
 21
 23
 11
 24
 27
 19
 16
 19
 15
 13
  9
  8
 17
 15
 10
 17
 16
 17
 19
 19
 16
 24
 11
 13
 19
 14
 35
 28
  7
 16
 18
 11
 20
 21
 38
  8
 25
 19
 34
 38
 55
 49
  8
 24
  8
 32
 25
 16
 31
  9
 27
 23
 23
 22
 17
 17
 27
 33
 35
 21
 11
 29
 13
 26
 41
 21
 32
 13
  8
 40
 23
 20
 23
 10
 14
 40
  6
 23
 20
 11
  7
 25
  5
 32
 60
  4
  6
 20
 13
 12
 42
 29
 29
 29


 23
 23
  8
 24
 25
 10
 24
  8
 17
 33
 13
 30
 14
 23
  8
 26
 18
 16
  9
 10
 31
 21
  4
 30
 25
 27
 35
 16
 13
 17
 31
  5
 13
 26
 16
 23
 35
 23
  6
 28
 15
 23
 11
 12
 21
 22
  6
 21
 30
 22
 19
 15
 24
 29
  6
 38
 19
 15
 23
 21
 32
 28
 17
 23
 12
 12
 13
 11
 35
 39
 28
 25
  7
 12
 16
 18
  3
 27
 47
 23
 24
 23
 37
 13
 12
 15
  9
 25
 40
 20
 32
 31
 40
  5
 41
 17
 25
 39
 18
 16
 33
 23
  9
 19
 20
  6
  9
 29
 10
 29
 16
  9
 11
 39
 14
 30
 13
 18
 18
 20
 16
 25
 26
 20
  5
 19
 25
 31
 21
 25
 26
 14
 12
 30
 14
 15
  5
 25
 26
 26
 16
 22
 22
 19
 30
 10
 23
 26
 40
 17
 25
 35
 33
 30
 36
 32
 31
 22
 32
 18
 37
 22
  9
 10
 18
 27
 17
 18
 25
 27
 22
 25
 28
 19
 29
 34
 20
 15
 18
 16
 12
 19
 15
 30
 13
 19
 23
 22
 15
 16
 25
 17
 22
 30
 11
 30
 17
 29
 23
 51
 33
 28
 13
 32
 11
 19
 21
 20
 27
 20
 19
 21
 23
 13
 42
 31
 13
 16
 32
 17
 14
 32
 29
 21
 13
 17
 25
 39
 18
 17
 31
 33
 35
  5
 31
 31
 20
 31
 13
 18
 19
 26
 18
 11
 31
 33
 24
 31
 23
 21


  6
  5
 10
  3
  8
  4
  9
  4
 14
  3
  8
  2
  8
  4
  7
  5
  7
  4
  8
  2
  6
  3
  9
  1
 14
  4
 15
  4
 13
  3
 33
  5
 11
  1
 20
  5
 15
  6
  8
  9
 10
 10
 13
  4
 14
  8
 16
 39
 24
  8
 30
  8
  4
  4
 16
 15
 28
 17
 25
 21
 16
 15
 15
 39
 25
 15
 26
 25
 18
 25
 29
 12
 15
 15
  2
 21
 14
 11
 12
 10
 16
 29
 19
 13
 29
 15
 31
 29
 18
  6
  2
 26
  8
  5
  2
  8
 13
 37
 15
 18
 16
 23
 30
 17
 30
 17
 13
 17
 18
 23
  5
 40
 16
 28
 37
 10
 23
 10
  2
 15
 20
 11
  7
 17
 10
 10
 19
 25
 16
 16
 17
 28
 12
 36
 25
  7
 27
 25
 14
  4
 30
 21
 17
 31
 18
 46
 30
  3
 29
 22
 12
 15
 13
 15
 12
 39
 13
 19
 38
 27
 16
  1
  1
 24
 14
 15
 24
 10
 29
  4
 29
 20
 41
 14
 22
 28
 31
 21
 15
 18
 21
 18
 17
 21
 16
 21
 30
 25
 11
  6
 20
 37
 16
 19
 25
 22
 28
 24
 15
 23
 54
 27
  9
 14
 24
 21
 19
 15
 24
 35
 10
 11
 13
 39
 15
 34
 19
 26
 24
 12
 29
 34
 17
 27
 25
  8
 16
 18
 26
  6
 19
 27
 11
 27
 11
 21
 25
  6
  5
 14
 22
 19
 21
  6
 14
  7
 24
 19
 11
 20


 27
 26
 32
 21
 20
 22
 25
 26
 27
 24
 33
 28
 24
 17
 22
 40
  8
 24
 28
 25
 40
 23
  6
 34
 24
 24
 39
 17
  5
 33
 20
 49
 22
 29
 22
 31
 20
 21
 18
 27
 14
 32
  2
 33
 12
 19
 24
 25
 29
 23
 19
 34
 22
 24
 20
 28
 12
 13
  9
 18
 17
 25
 14
 19
 23
 34
 29
 24
 20
 25
 18
 16
 22
 31
 13
 14
 21
  5
 15
 24
 23
 15
 22
  6
 36
 17
 30
 28
 11
 13
 29
 33
 25
  6
  5
 35
 17
 33
 24
 31
 35
 14
 28
 16
 18
 17
 18
 12
 19
 18
 39
  8
 36
 33
 10
  6
 18
 13
 18
 15
  6
 19
  6
 26
 11
 21
  8
 24
 31
 33
 20
 17
 21
 24
 27
  8
 12
 56
 12
 35
 30
 15
 23
  8
 26
 41
  8
 18
 13
 59
 21
 17
 15
 36
 17
 19
 38
 35
 41
 18
 38
 22
 23
 26
 22
 14
 11
 18
 14
 18
 16
 29
 25
 30
 28
 12
 10
 22
 33
  2
  8
 29
 14
 19
 23
 24
 18
 10
 18
 12
  8
  8
 23
 14
 27
  5
 21
 19
 25
 26
 15
 13
 23
 10
  9
 19
 13
 13
 19
  8
 18
  1
  4
 14
  7
  7
  9
 39
 33
 25
 31
 22
 24
 23
 12
 28
 14
 23
 15
 32
 47
 30
 22
 32
 10
 24
 15
 16
 25
 31
 25
 15
 22
 13
 24
 37
 13
  2
 19
 16


{
  1 : DoubleTensor - size: 1x2
  2 : DoubleTensor - size: 1x2
  3 : DoubleTensor - size: 1x2
  4 : DoubleTensor - size: 1x2
  5 : DoubleTensor - size: 1x2
  6 : DoubleTensor - size: 1x2
  7 : DoubleTensor - size: 1x2
  8 : DoubleTensor - size: 1x2
  9 : DoubleTensor - size: 1x2
  10 : DoubleTensor - size: 1x2
}


bad argument #2 to '?' (out of range at /Users/austinshin/torch/pkg/torch/generic/Tensor.c:890)
stack traceback:
	[C]: at 0x0cf96ad0
	[C]: in function '__index'
	[string "function predict(input_set, output_set, cutof..."]:11: in function 'f'
	[string "local f = function() return predict(valid_kag..."]:1: in main chunk
	[C]: in function 'xpcall'
	...s/austinshin/torch/install/share/lua/5.1/itorch/main.lua:209: in function <...s/austinshin/torch/install/share/lua/5.1/itorch/main.lua:173>
	...s/austinshin/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...ustinshin/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...ustinshin/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...ustinshin/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	...s/austinshin/torch/install/share/lua/5.1/itorch/main.lua:381: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x010cc0dbc0: 